In [1]:
#|default_exp img_sen2

# Sentinel 2

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee
import geemap
from datetime import datetime

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils
from gee_projects import const_vars as const
from gee_projects import mask_s2 as mask_s2

sentinel-2 class

In [4]:
#|export

class access_s2_image():
  
  def __init__(self,
               path_to_geometry : str , # path to geometry file, can be .shp or .gpkg . example : r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg"
               crs : str , #coordinate reference system to work with
               start_date : str , # Start date for searching images. , 
               end_date : str , # End date for searching images.

               one_day_image : bool == True , # when True, image will be generated for the neares date to the provided user_date, else, median image between the two dates will be generated. 
               user_date : str==None , # The user-provided date of interest; the script accesses the image with the closest available date to the provided date
               ):
    
    if one_day_image and user_date is None:
      raise ValueError("Error:if one_day_image is True, user_date cannot be None. Please insert a date or change one_day_image to False to get aggregated image.")
        
    self.crs = crs
    self.fc = utils.gdf_to_featureCollection(
      utils.read_gdf(geometry_path=path_to_geometry,crs=self.crs))
    self.start_date=start_date
    self.end_date=end_date
    self.one_day_image = one_day_image
    self.user_date = user_date
    
    self.fc = utils.gdf_to_featureCollection(utils.read_gdf
                                        (geometry_path=path_to_geometry,crs=self.crs))
    
    #pre process s2
    s2_sr_cld_col_eval = mask_s2.get_s2_sr_cld_col(self.fc, self.start_date, self.end_date)

    self.s2_sr = (s2_sr_cld_col_eval.map(mask_s2.add_cld_shdw_mask)
             .map(mask_s2.apply_cld_shdw_mask))

    if one_day_image:

      #get list of available dates of the image collection
      self.available_dates = utils.list_dates(self.s2_sr)
      self.closest_date = utils.find_closest_date(date_list = self.available_dates,
                                                  user_date=self.user_date)
      
      #filter image collection to get the image from the closest date :
      print(f'Requested date: {self.user_date}\n' 
            f'The nearest date available: {self.closest_date}')
      
      day_before_day_after = utils.get_day_before_day_after(self.closest_date)
      self.img = self.s2_sr.filterDate(day_before_day_after[0], day_before_day_after[1]).mosaic().clip(self.fc)
    
    else:
      self.img = self.s2_sr.median().clip(self.fc)


In [6]:
extractor = access_s2_image(path_to_geometry=r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg", 
                            crs = "4326",
                            start_date = '2023-10-06',
                            end_date = '2023-10-08',
                            one_day_image = True,
                            user_date = '2023-10-07')

# extractor.img.bandNames().getInfo()
extractor.img

Requested date: 2023-10-07
The nearest date available: 2023-10-07


In [ ]:
Map = geemap.Map()

Map.add_basemap("SATELLITE")

# Viz = {'min': 0, 'max': 5000, 'palette': [    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
#                                          '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
#                                          '012E01', '011D01', '011301']}

Map.addLayer(extractor.img.select(['B4','B3','B2']), {}, "img1")

Map.centerObject(extractor.fc)
Map